# Perform lever-arm correction to all sensors from Sentry Nav

Sentry measured rotations relate directly to local spike translations.
- Pitching motion (about Sentry y-axis) relates to vertical motions in the spike.
- Bottom following autonomous mode will result in many pitching adjustments
 - Likely a major vertical component measured in ADV 

In [ ]:
# Libraries for plotting
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# load processed Nav data
import pandas as pd
nav = pd.read_hdf('../data/interim/nav_converted.h5', 'table')
paros = pd.read_hdf('../data/interim/paros_converted.h5', 'table')
sbe3 = pd.read_hdf('../data/interim/sbe3_converted.h5', 'table')

In [ ]:
# get start time
T0 = nav.head(1).epoch[0]

# see what is in the data
nav.head(2)

## Lever-arm uses the relative position from reference to test point

In [ ]:
# Sentry to spike GX3-25,  this is the guesstimate, will look up more accurate number 
bTRgxs = np.array([1.8, 0.2, 0.7])
# Sentry to body GX3-25,  also a guesstimate
bTRgxb = np.array([0.0, 0.0, 0.0])
# Sentry to ADV,  also a guesstimate
bTRadv = np.array([1.6, 0.2, 0.9])
# Sentry to SBE3,  also a guesstimate
bTRsbe = np.array([1.5, 0.2, 0.8])
# Sentry to Paros,  also a guesstimate
bTRpar = np.array([1.4, 0.2, 0.8])

## Extract time and pitch

Convert pitch from degrees to radians

In [ ]:
# Data for plotting
t = nav.epoch - T0
pitch = nav.pitch * np.pi/180.0
nav["pitch_rad"] = nav.pitch * np.pi/180.0

### Double checking epoch time?

There is a jump in timestamp?

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(len(t)),t)

ax.set(xlabel='samples', ylabel='epoch time',
       title='Data from two dives?')
ax.grid()
fig.savefig('../results/nav/epoch-time.png')

## Working with a shorter subset of data

In [ ]:
strt = 0
ephs = 65000

In [ ]:
fig, ax = plt.subplots()
ax.plot(t[strt:ephs], pitch[strt:ephs])

ax.set(xlabel='time [s]', ylabel='pitch [rad]',
       title='Pitch angle')
ax.grid()
fig.savefig('../results/nav/pitch_vs_time.png')

# Calculate lever-arm perturbations to Spike mounted sensors

In [ ]:
fig, ax = plt.subplots()
ax.plot(t[strt:ephs], nav.head(ephs).depth)

ax.set(xlabel='time [s]', ylabel='depth [m]',
       title='Depth over time')
ax.grid()
fig.savefig('../results/nav/depth_vs_time.png')

## Seems to be some erroneous data between 65000 and 70000

In [ ]:
fig, ax = plt.subplots()
ax.plot(t[65000:70000], nav.head(70000).tail(5000).depth)

ax.set(xlabel='time [s]', ylabel='depth [m]',
       title='Depth Measurement Error')
ax.grid()
fig.savefig('../results/nav/depth_data_err.png')

## Zoom in closer on the first dive

In [ ]:
strt = 14000
ephs = 50500

In [ ]:
fig, ax = plt.subplots()
ax.plot(t[strt:ephs], nav.head(ephs).tail(ephs-strt).depth)

ax.set(xlabel='time [s]', ylabel='depth [m]',
       title='Depth over time')
ax.grid()
fig.savefig('../results/nav/depth_vs_time_bottom1_all.png')

In [ ]:
strt = 14780
ephs = 25330

In [ ]:
fig, ax = plt.subplots()
ax.plot(t[strt:ephs], nav.head(ephs).tail(ephs-strt).depth)

ax.set(xlabel='time [s]', ylabel='depth [m]',
       title='Depth Zoomed, Bottom1, -1480m')
ax.grid()
fig.savefig('../results/nav/depth_vs_time_bottom1_1480.png')

In [ ]:
fig, ax = plt.subplots()
ax.plot(t[strt:ephs], pitch[strt:ephs])

ax.set(xlabel='time [s]', ylabel='pitch [rad]',
       title='Pitch angle, Bottom1, -1480m')
ax.grid()
fig.savefig('../results/nav/pitch_vs_time_bottom1_1480.png')

### What are the Local Vertical Perturbations due to Pitching

In [ ]:
def getpitchrate(pitch_rad):
    # focus on just pitch for the moment
    local_pitch = np.array(pitch_rad)
    ref_pitch = np.mean(local_pitch)
    delta_pitch = local_pitch - ref_pitch
    # calculate the delta time
    local_time = np.array(t)
    delta_time = np.diff(local_time)
    # pitch rate
    local_pitch_rate = delta_pitch / np.mean(delta_time)
    return local_pitch_rate

In [ ]:
local_pitch_rate = getpitchrate(pitch[strt:ephs])

In [ ]:
fig, ax = plt.subplots()
ax.plot(local_pitch_rate)

ax.set(xlabel='time [s]', ylabel='pitch rate [rad/s]',
       title='Pitch rate, Bottom1, -1480m')
ax.grid()
fig.savefig('../results/nav/pitchrate_vs_time_bottom1_1480.png')

In [ ]:
# calculate the lever-arm component of vertical velocity

# result
res = np.matrix((10550,3))
vert = np.zeros((10550,))

vert.shape

In [ ]:
for i in range(10550):
  local_rate = np.array([0.0, local_pitch_rate[i], 0.0])
  val = np.cross(local_rate, bTRadv)
  vert[i] = val[2]

In [ ]:
fig, ax = plt.subplots()
ax.plot(vert)

ax.set(xlabel='time [s]', ylabel='Vertical Vel [ms/s]',
       title='Vertical Velocity, Bottom1, -1480m')
ax.grid()
fig.savefig('../results/nav/vert_vs_time_bottom1_1480.png')

# Use Panda's Data Selection Methodology

### Study comparison between Nav lever-arm and Paros

In [ ]:
#def area_select(df):
#    return df.loc[(df.xrot >= xmin) & (df.xrot <= xmax) & (df.yrot >= ymin) & (df.yrot <= ymax)]

def sel_by_depth(nav, depthmin=-5000, depthmax=-1000):
  return nav[(nav.depth < depthmax) & (nav.depth > depthmin)]

def sel_by_epoch(nav, T0, Tmin=-5000, Tmax=-1000):
  return nav[(nav.epoch < (Tmax+T0)) & (nav.epoch > (Tmin+T0))]

In [ ]:
sel_depth = sel_by_depth(nav, depthmin=-1482, depthmax=-1475)

fig, ax = plt.subplots()
ax.plot(sel_depth.depth.values)

ax.set(xlabel='samples', ylabel='depth [m]',
       title='Depth Zoomed, Bottom1, -1480m')
ax.grid()
fig.savefig('../results/nav/sel_depth_1480.png')

In [ ]:
sel_depth = sel_by_depth(nav, depthmin=-3000, depthmax=-1400)

fig, ax = plt.subplots()
ax.plot(sel_depth.depth.values)

ax.set(xlabel='samples', ylabel='depth [m]',
       title='Depth Zoomed, Bottom1, -1480m')
ax.grid()
fig.savefig('../results/nav/sel_depth_dives.png')

In [ ]:
paros.head(2)

In [ ]:
sel_depth = sel_by_depth(nav, depthmin=-1482, depthmax=-1475)
sel_depth_t = sel_by_epoch(sel_depth, T0, Tmin=2900, Tmax=6000)

fig, ax = plt.subplots()
ax.plot(sel_depth_t.epoch.values - T0, sel_depth_t.depth.values)

ax.set(xlabel='epoch [s]', ylabel='depth [m]',
       title='Depth Zoomed, Bottom1, -1480m')
ax.grid()
fig.savefig('../results/nav/sel_depth_time_01.png')

In [ ]:
sel_paros_t = sel_by_epoch(paros, T0, Tmin=2950, Tmax=5080)

fig, ax = plt.subplots()
ax.plot(sel_paros_t.epoch.values - T0, sel_paros_t.pressure.values)

ax.set(xlabel='time [s]', ylabel='depth [pressure]',
       title='Depth Zoomed, Bottom1, -1480m')
ax.grid()
fig.savefig('../results/nav/sel_paros_first.png')

In [ ]:
def first_order_comp(x, m, c):
    return x*m + c

In [ ]:
sel_paros_t = sel_by_epoch(paros, T0, Tmin=2950, Tmax=5080)
sel_depth_t = sel_by_epoch(sel_depth, T0, Tmin=2950, Tmax=5080)

fig, ax = plt.subplots()
ax.plot(sel_paros_t.epoch.values - T0, first_order_comp(sel_paros_t.pressure.values,-1,707))
ax.plot(sel_depth_t.epoch.values - T0, sel_depth_t.depth.values)

ax.set(xlabel='time [s]', ylabel='depth ',
       title='Depth Zoomed, Bottom1, -1480m')
ax.grid()
fig.savefig('../results/nav/sel_paros_prefit.png')

In [ ]:
sel_sbe3_t = sel_by_epoch(sbe3, T0, Tmin=2950, Tmax=5080)

fig, ax1 = plt.subplots()
hd1, = ax1.plot(sel_paros_t.epoch.values - T0, first_order_comp(sel_paros_t.pressure.values,-1,707), label='nav')
hd2, = ax1.plot(sel_depth_t.epoch.values - T0, sel_depth_t.depth.values,'r', label='paros')

ax1.set(xlabel='time [s]', ylabel='depth [m]',
       title='Zoomed depth & temperature, bottom01, -1480m')
ax1.grid()

ax2 = ax1.twinx()
hd3, = ax2.plot(sel_sbe3_t.epoch.values - T0, sel_sbe3_t.temp_stinger.values, 'm', label='sbe3')
ax2.set_ylabel('temp_stinger', color='m')
ax2.tick_params('y', colors='m')

fig.tight_layout()
# plt.show()
plt.legend(handles=[hd1, hd2, hd3])

fig.savefig('../results/nav/bottom01_depth_temp.png')

In [ ]:
sbe3.head(2)

## Fit Paros to Nav

Extract a slice of Nav and Paros data, then interpolate paros to nav followed by a first order fit.

In [ ]:
nav_slice = sel_by_depth(nav, depthmin=-1550, depthmax=-1450).copy()
nav_slice['pressure'] = np.interp(nav_slice.epoch.values, paros.epoch.values, paros.pressure.values)
params = np.polyfit(nav_slice.pressure.values, nav_slice.depth.values, 1)
print('Fit Parameters: %0.10f %0.10f' % (params[0], params[1]))

In [ ]:
# add paros predicted depth
nav_slice['paros_depth'] = nav_slice.pressure*params[0] + params[1]
nav_slice.head(1)

In [ ]:
nav_slice_plot = sel_by_epoch(nav_slice, T0, Tmin=2960, Tmax=5070)

fig, ax1 = plt.subplots(figsize=(12, 9), dpi=80)
hd1, = ax1.plot(nav_slice_plot.epoch.values - T0, nav_slice_plot.depth.values, label='nav')
hd2, = ax1.plot(nav_slice_plot.epoch.values - T0, nav_slice_plot.paros_depth.values,'r', label='paros')

ax1.set(xlabel='time [s]', ylabel='depth [m]',
       title='Zoomed depth & temperature, bottom01, -1480m')
ax1.grid()

# ax2 = ax1.twinx()
# hd3, = ax2.plot(nav_slice.epoch.values - T0, sel_sbe3_t.temp_stinger.values, 'm', label='sbe3')
# ax2.set_ylabel('temp_stinger', color='m')
# ax2.tick_params('y', colors='m')

# fig.tight_layout()
# plt.show()
plt.legend(handles=[hd1, hd2])

fig.savefig('../results/nav/firstfit_paros_to_nav.png')

In [ ]:
nav_slice_plot.pitch_rad.values

In [ ]:
local_pitch_rate = getpitchrate(nav_slice_plot.pitch_rad)

In [ ]:
fig, ax = plt.subplots()
ax.plot(local_pitch_rate)

ax.set(xlabel='time [s]', ylabel='pitch rate [rad/s]',
       title='Pitch rate, Bottom1, -1480m')
ax.grid()
fig.savefig('../results/nav/localpitchrate_1480.png')

In [ ]:
def rate2vel_pitch(rate):
    numelems = len(rate)
    res = np.matrix((numelems,3))
    vert = np.zeros((numelems,))
    for i in range(numelems):
        local_rate = np.array([0.0, rate[i], 0.0])
        val = np.cross(local_rate, bTRadv)
        vert[i] = val[2]
    return vert

In [ ]:
vert = rate2vel_pitch(local_pitch_rate)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9), dpi=80)
ax.plot(vert2)

ax.set(xlabel='time [s]', ylabel='Vertical Vel [m/s]',
       title='Vertical Velocity, Bottom1, -1480m')
ax.grid()
fig.savefig('../results/nav/pitchleverarm_1480.png')

In [ ]:
vert_offset = np.cumsum(vert)

fig, ax = plt.subplots()
ax.plot(vert_offset)

ax.set(xlabel='samples', ylabel='vertical offset [m]',
       title='Vertical Offset, Bottom1, -1480m')
ax.grid()

In [ ]:
# add pitching offset onto paros
derotparos_depth = nav_slice_plot.paros_depth.values - vert_offset

# Pitch biasing question

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 9), dpi=80)
hd1, = ax1.plot(nav_slice_plot.epoch.values - T0, nav_slice_plot.paros_depth.values,'r', label='paros')
hd2, = ax1.plot(nav_slice_plot.epoch.values - T0, nav_slice_plot.depth.values, label='nav')
hd3, = ax1.plot(nav_slice_plot.epoch.values - T0, derotparos_depth,'g', label='paros-arm')


ax1.set(xlabel='time [s]', ylabel='depth [m]',
       title='Zoomed depth & temperature, bottom01, -1480m')
ax1.grid()

# ax2 = ax1.twinx()
# hd4, = ax2.plot(nav_slice_plot.epoch.values - T0, vert_offset, 'm', label='arm-displ')
# ax2.set_ylabel('Vertical Offset', color='m')
# ax2.tick_params('y', colors='m')

# fig.tight_layout()
# plt.show()
plt.legend(handles=[hd1, hd2, hd3])

fig.savefig('../results/nav/compare_verts.png')

# New Vertical Velocities

Still pending large lever-arm offset question  

In [ ]:
dt = np.diff(nav_slice_plot.epoch)
nav_vertvel = np.diff(nav_slice_plot.depth)/dt - 3.0
paros_vertvel = np.diff(nav_slice_plot.paros_depth)/dt
parosarm_vertvel = np.diff(derotparos_depth)/dt + 3.0

# nav-arm
navarm_vertvel = vert - 5


fig, ax1 = plt.subplots(figsize=(12, 9), dpi=80)
hd1, = ax1.plot(nav_vertvel,'r', label='nav')
hd2, = ax1.plot(paros_vertvel, 'b', label='paros')
hd3, = ax1.plot(parosarm_vertvel,'g', label='paros-arm')
hd4, = ax1.plot(navarm_vertvel,'c', label='nav-arm')


ax1.set(xlabel='samples', ylabel='vertical vel [m/s]',
       title='Zoomed depth & temperature, bottom01, -1480m')
ax1.grid()
plt.legend(handles=[hd1, hd2, hd3, hd4])